In [ ]:
!pip install pdfplumber
!pip install stanza

Kütüphaneler

In [406]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import re
import pdfplumber
import json
import re
import string
import stanza

PDF Çevirme İşlemi

In [407]:
text = ""
with pdfplumber.open("/content/drive/MyDrive/case9.pdf") as pdf:
    for page in pdf.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text.replace('\n', ' ')

print(text)

Sayın İlgili, 1 Haziran'da gönderdiğim kargo hâlâ elime ulaşmadı. Kargo takip sisteminde sürekli "yolda" ifadesi yer almakta. Müşteri hizmetlerinden de tatmin edici bir yanıt alamadım. Mağduriyetimin giderilmesini ve açıklama yapılmasını talep ediyorum. Caner Arslan, Ortahisar / Trabzon


In [408]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



Modeller
1.  BERTurk (savasy): Bu model kişi adı soyadı, adres, tarih ve kurum gibi bilgi çıkarımı için kullanılmıştır.
2.  Stanza (Stanford): Lemmatizationişlemi için kullanılmıştır.
3.  Zero-Shot-Classification: Olay konusu ve talep için kullanılmıştır.
4. Text-Classification: Dilin Resmiyetininde argo kelimeleri tespit etmek için kullanılmıştır.



In [409]:
# NER modeli: BERTurk (savasy)
model_name = "savasy/bert-base-turkish-ner-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

#Zero-Shot-Classification(RoBERTa tabanlı)
classifier = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli")

#Ner modeli: Stanza
stanza.download('tr')
nlp = stanza.Pipeline('tr')

#Text-Classification
nlp1 = pipeline("text-classification", model="fc63/toxic-classification-model", trust_remote_code=True)

Some weights of the model checkpoint at savasy/bert-base-turkish-ner-cased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint o

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: tr (Turkish) ...
INFO:stanza:File exists: /root/stanza_resources/tr/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: tr (Turkish):
| Processor | Package       |
-----------------------------
| tokenize  | imst          |
| mwt       | imst          |
| pos       | imst_charlm   |
| lemma     | imst_nocharlm |
| depparse  | imst_charlm   |
| ner       | starlang      |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!
Device set to use cpu


def lemmatization(text):
    doc = nlp(text)
    lemmas = []
    for sentence in doc.sentences:
        for word in sentence.words:
            lemmas.append(word.lemma.lower())
    return ' '.join(lemmas)

Açık Bilgi Çıkarımı

In [411]:
def event_category(text):
    labels = [
        "çöpler", "asansör", "su", "elektrik","internet","otopark","alt yapı",
        "gürültü", "aydınlatma", "ısınma", "trafik", "kargo","ulaşım", "yol"
    ]
    result = classifier(text, candidate_labels=labels)
    return result["labels"][0], result["scores"][0]


def request_category(text):
    labels = [
        "Çözüm Talebi",
        "Açıklama Talebi",
        "Denetim Talebi"
    ]
    result = classifier(text, candidate_labels=labels)
    best = result["labels"][0]
    return best

In [ ]:
def extract_all_info(text):
    info = {
        "Ad Soyad": None,
        "Adres veya bölge bilgisi": "",
        "Tarih aralıkları": [],
        "Kurum ismi": [],
        "Olay konusu": None,
        "Talep": None
    }

    text_lower = text.lower()
    entities = ner_pipeline(text)

    aylar = ["ocak", "şubat", "mart", "nisan", "mayıs", "haziran","temmuz", "ağustos", "eylül", "ekim", "kasım", "aralık"]

    # 1. NER tabanlı çıkarım 
    for ent in entities:
        label = ent["entity_group"]
        word = ent["word"].strip()

        if label == "PER" and not info["Ad Soyad"]:
            info["Ad Soyad"] = word

        elif label == "MISC":
            if any(ay in word.lower() for ay in aylar):
                 info["Tarih aralıkları"].append(word)

        elif label == "LOC":
            if word.lower() not in aylar and word not in info["Adres veya bölge bilgisi"]:
                 info["Adres veya bölge bilgisi"] += word + " "

        elif label == "ORG":
            if word and word.lower() not in aylar and word not in info["Kurum ismi"]:
                info["Kurum ismi"].append(word)

    # 2. RegEx ile zaman ifadeleri
    date_pattern = (
        r"\b(?:bugün|dün|şimdi|az önce|geçmişte|gelecekte|"
        r"hafta sonları|hafta içleri|"
        r"(?:\d+|bir|iki|üç|dört|beş|altı|yedi|sekiz|dokuz|on)?\s*(?:gün|hafta|ay|yıl)"
        r"(?:dır|dir|den beri|önce)?|"
        r"(?:yaz|kış|ilkbahar|sonbahar)(?:da|de|aylarında|dönemi)?|"
        r"geceleri|gündüzleri|geçen ay|geçtiğimiz ay|gelecek yıl|bu yıl|bu hafta|"
        r"(?:yağmurlu|güneşli|karlı|fırtınalı|sisli)\s*(?:günlerde|sabahlarda|akşamlarda|zamanlarda)?|"
        r"\d{1,2}\s+(?:ocak|şubat|mart|nisan|mayıs|haziran|temmuz|ağustos|eylül|ekim|kasım|aralık)|"
        r"son \d+ gün"
        r")\b"
    )
    matches = list(re.finditer(date_pattern, text_lower, re.IGNORECASE))

    info["Tarih aralıkları"] = []
    if matches:
        for m in matches:
            info["Tarih aralıkları"].append(m.group(0).strip())
    else:
        info["Tarih aralıkları"].append("Genel")

    # 3. RegEx ile kurum İFADELERİ 
    lemmatized_text = lemmatization(text)
    organisation = [
        "valilik", "kaymakamlık", "belediye", "belediye başkanlığı", "müdürlük",
        "bakanlık", "üniversite", "fakülte", "kurum", "daire başkanlığı",
        "il müdürlüğü", "genel müdürlük", "başkanlık", "rektörlük",
        "yetkili", "birim", "ilgili birim",  "idare", "otorite", "ilgili"
    ]
    for keyword in organisation:
        if keyword in lemmatized_text and keyword not in info["Kurum ismi"]:
            info["Kurum ismi"].append(keyword)

    # 4. Zero-Shot olay kategorisi 
    try:
        event, score = event_category(text)
        if score > 0.3:
            info["Olay konusu"] = event
    except:
        info["Olay konusu"] = None

    # 5. Zero-Shot talep kategorisi
    try:
        request = request_category(text)
        if request:
            info["Talep"] = request
    except:
        info["Talep"] = None

    info["Tarih aralıkları"] = list(set([d.strip() for d in info["Tarih aralıkları"] if d.strip()]))
    info["Kurum ismi"] = list(set([k.strip() for k in info["Kurum ismi"] if k.strip()]))
    info["Adres veya bölge bilgisi"] = info["Adres veya bölge bilgisi"].strip()

    return info

JSON Kaydetme İşlemi

In [ ]:
info = extract_all_info(text)

with open("dilekce_bilgisi.json", "w", encoding="utf-8") as f:
    json.dump(info, f, ensure_ascii=False, indent=4)

Yorumla Anlam Çıkarma

In [ ]:
def classify_issue(cumle):
    kurallar = {
        "Tekrarlayan sorun": [
            r"\bher\s+(akşam|sabah|gün|gündüz|gece|pazartesi|salı|çarşamba|perşembe|cuma|cumartesi|pazar|hafta içi|hafta sonu|ay)\b",
            r"\byine\b",
            r"\btekrar\b"
        ],
        "Uzun süredir devam eden sorun": [
            r"\bdefalarca\b",
            r"\bçok\s+kez\b",
            r"\bbirçok\s+kez\b",
            r"\bkaç\s+kere\b"
        ],
        "Eksiklik Bildirimi": [
            r"\beksik", r"\byetersiz", r"\bihtiyaç",r"\bmevcut değil",r"\bkontrol",r"\bulaşmadı",
            r"\bbulunmuyor", r"\bhiç yok", r"\baz sayıda",r"\bbakım",r"\bonarım"
        ],
       "Ciddi bir endişe olan sorun": [
            r"\bkorku", r"\bendişe", r"\bhasta",r"\bciddi",
            r"\brisk", r"\brahatsız", r"\btedirgin",r"\btartışma",
            r"\btehlike", r"\bacilen", r"\bkötü",r"\bgüvensiz",
            r"\bsağlık", r"\bnefes alamıyor", r"\bfiziksel engel", r"\bzor durum",
            r"\btedavi", r"\bpanik", r"\büzülmek", r"\bstres", r"\btehdit",
            r"\bçaresiz"
        ]

    }

    anlamlar = []
    for anlam, kaliplar in kurallar.items():
        for kalip in kaliplar:
            if re.search(kalip, text, re.IGNORECASE):
                anlamlar.append(anlam)
                break

    return anlamlar

Ton Analizi

In [415]:
def language_tone_analysis(text):
    text = text.lower()

    angry_keywords = [
        "artık yeter", "kabul edilemez", "şikayetçiyim", "çok kötü", "rahatsız oldum",
        "bıktım", "sinirliyim", "tepki", "şiddetli", "zorbalık", "asla kabul", "adaletsiz"
    ]

    desperate_keywords = [
        "çaresiz", "yardım edin", "yapacak bir şey yok", "umudum kalmadı",
        "endişeliyim", "umutsuz", "tedirginim", "çözüm bulunmadı", "çözüm yok"
    ]

    formal_keywords = [
    "sayın", "makamına", "ilgili birim", "arz ederim", "gereği",
    "kurumunuza", "değerli", "hususunda", "dilekçe", "resmî"
    ]

    polite_complaint_keywords = [
        "rica ediyorum", "talep ediyorum", "yardımcı olur musunuz",
        "gereğinin yapılmasını", "bekliyorum", "bilgi verir misiniz", "yardımcı olur musunuz"
    ]


    if any(keyword in text for keyword in angry_keywords):
        return "Öfkeli / Tepkili"

    if any(keyword in text for keyword in desperate_keywords):
        return "Çaresiz"

    if any(keyword in text for keyword in formal_keywords):
        return "Nesnel"

    if any(keyword in text for keyword in polite_complaint_keywords):
        return "Kibar Şikâyet"

    return "Sakin"


Dil Analizi

In [416]:
def is_formal_language(text):
    text = text.lower()
    header_pattern = r'\b(sayın|değerli)\b'
    footer_pattern = r'\b(arz ederim|arz ediyorum|rica ediyorum|talep ederim|talep ediyorum)\b'
    has_header = re.search(header_pattern, text) is not None
    has_footer = re.search(footer_pattern, text) is not None
    return has_header and has_footer

nlp1 = pipeline("text-classification", model="fc63/toxic-classification-model", trust_remote_code=True)

def is_offensive_or_toxic(text, threshold=0.7):
    result = nlp1(text)[0]
    label = result["label"].lower()
    score = result["score"]
    return (label in ["toxic", "label_1"] and score >= threshold), score

def language_formality_analysis(text):
    if is_formal_language(text):
        return "Resmi"
    elif is_offensive_or_toxic(text)[0]:
        return "Argo"
    else:
        return "Samimi"

Device set to use cpu


Yazım Kuralları

In [417]:
def check_spelling(text):
    error_score = 0

    if not text[0].isupper():
        error_score += 1

    if not re.search(r'[.!?]$', text.strip()):
        error_score += 1

    if re.search(r'[.!?,;:]{3,}', text):
        error_score += 1

    if re.search(r'([.,:;!?])[^ \n]', text):
        error_score += 1

    entities = ner_pipeline(text)
    for ent in entities:
        label = ent["entity_group"]
        word = ent["word"].strip()
        if label in ["PER", "ORG", "LOC"]:
            if word and word[0].isalpha() and not word[0].isupper():
                error_score += 1

    sentences = re.split(r'(?<=[.!?:])\s+', text.strip())
    for sentence in sentences[1:]:
        if sentence and not sentence[0].isupper():
            error_score += 1

    if error_score <= 1:
        return "Yüksek"
    elif error_score <= 3:
        return "Orta"
    else:
        return "Düşük"

Sonuç Gösterimi

In [418]:
def show_result(text):
    info = extract_all_info(text)
    meaning_extraction = classify_issue(text)
    tone_analysis = language_tone_analysis(text)
    language_formality = language_formality_analysis(text)
    spelling_check = check_spelling(text)

    print("\n--- DİLEKÇE ANALİZİ ---")
    print(f"Ad Soyad                : {info.get('Ad Soyad')}")
    print(f"Adres                   : {info.get('Adres veya bölge bilgisi')}")
    print(f"Zaman                   : {', '.join(info.get('Tarih aralıkları', []))}")
    print(f"Kurum                   : {', '.join(info.get('Kurum ismi', []))}")
    print(f"Olay Konusu             : {info.get('Olay konusu')}")
    print(f"Talep                   : {info.get('Talep')}")
    print(f"Anlam Çıkarımı          : {meaning_extraction}")
    print(f"Ton                     : {tone_analysis}")
    print(f"Dilin Resmiyeti         : {language_formality}")
    print(f"Yazım Kurallarına Uyum  : {spelling_check}")


In [419]:
print(show_result(text))


--- DİLEKÇE ANALİZİ ---
Ad Soyad                : Caner Arslan
Adres                   : Ortahisar Trabzon
Zaman                   : 1 haziran
Kurum                   : ilgili
Olay Konusu             : ulaşım
Talep                   : Açıklama Talebi
Anlam Çıkarımı          : ['Eksiklik Bildirimi']
Ton                     : Nesnel
Dilin Resmiyeti         : Resmi
Yazım Kurallarına Uyum  : Yüksek
None
